In [88]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import imodelsx.process_results
import sys
sys.path.append('../experiments/')
results_dir = '../results/01_train_model'
experiment_filename = '../experiments/01_train_model.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2093/2093 [00:33<00:00, 61.59it/s]


experiment varied these params: ['dataset_name', 'model_name', 'featurizer_name', 'featurizer_frac', 'depth', 'bit', 'max_depth', 'max_rules', 'max_trees', 'pre_interaction', 'pre_max_features', 'post_max_features', 'gpu']


dataset_name  model_name  featurizer_name  featurizer_frac  depth  bit  max_depth  max_rules  max_trees  pre_interaction  pre_max_features  post_max_features  gpu
abalone       figs        featurizer       0.3              2      0    1          50         30         l0l2             0.0               0.0                1      1
                                                                                   60         30         l0l2             0.0               0.0                1      1
                                                                   1    1          50         30         l0l2             0.0               0.0                1      1
                                                                                   60         30         l0l2             0.0               0.0                1      1
                                                            3      0    1          50         30         l0l2             0.0               0.0                1     

In [90]:
r_feat = r[r['featurizer_name'] == 'featurizer']
r_nfeat = r[r['featurizer_name'] == 'no_featurizer']

In [111]:
r_feat = r_feat.drop(columns=['n_epochs', 'n_epochs', 'gpu', 'size_interactions', 'use_cache', 'cat_mappings', 'task_type', 'save_dir', 
                     'seed', 'subsample_frac', 'featurizer_overlap', 'save_dir_unique'])

r_nfeat = r_nfeat.drop(columns=['n_epochs', 'n_epochs', 'gpu', 'size_interactions', 'use_cache', 'cat_mappings', 'task_type', 'save_dir', 
                     'seed', 'subsample_frac', 'featurizer_overlap', 'save_dir_unique'])

In [114]:
pd.pivot_table(r_feat[r_feat['bit']==0], values='r2_score_val_true', aggfunc='max', index='dataset_name', columns=['model_name']).round(2).dropna()

model_name,figs,ft_distill,ft_transformer,random_forest,resnet,rf_plus,xgboost
dataset_name,,,,,,,
abalone,0.41,0.42,0.36,0.44,0.35,0.45,0.42
airfoil,0.63,0.34,0.22,0.33,0.33,0.46,0.78
ca_housing,0.70,0.68,0.69,0.58,0.65,0.66,0.73
cpu_act,0.97,0.96,0.97,0.95,0.94,0.97,0.97
insurance,0.79,0.77,0.71,0.79,0.63,0.80,0.80
parkinsons,0.92,0.89,0.69,0.53,0.72,0.57,0.92
powerplant,0.93,0.93,0.92,0.91,0.90,0.93,0.94


In [115]:
pd.pivot_table(r_feat[r_feat['bit']==1], values='r2_score_val_true', aggfunc='max', index='dataset_name', columns=['model_name']).round(2).dropna()

model_name,figs,ft_distill,ft_transformer,random_forest,resnet,rf_plus,xgboost
dataset_name,,,,,,,
abalone,0.43,0.47,0.37,0.45,0.37,0.46,0.43
airfoil,0.77,0.62,0.48,0.60,0.46,0.64,0.86
ca_housing,0.72,0.68,0.67,0.61,0.66,0.67,0.73
concrete,0.84,0.74,0.67,0.77,0.63,0.83,0.86
cpu_act,0.97,0.97,0.97,0.96,0.95,0.97,0.98
insurance,0.80,0.86,0.77,0.86,0.61,0.86,0.82
miami_housing,0.84,0.77,0.80,0.78,0.79,0.79,0.88
parkinsons,0.91,0.82,0.74,0.83,0.55,0.83,0.90
powerplant,0.94,0.93,0.93,0.93,0.91,0.93,0.94


In [127]:
pd.pivot_table(r_nfeat, values='r2_score_val_true', aggfunc='mean', index='dataset_name', columns=['model_name']).round(2).dropna().to_csv('train.csv')

In [128]:
r_nfeat.to_csv('train.csv')

In [126]:
pd.pivot_table(r_nfeat, values='r2_score_val_true', aggfunc='median', index='dataset_name', columns=['model_name']).round(2).dropna()

model_name,figs,ft_transformer,random_forest,resnet,rf_plus,xgboost
dataset_name,,,,,,
abalone,0.33,0.45,0.50,0.43,0.53,0.48
airfoil,0.83,0.37,0.67,0.37,0.68,0.91
ca_housing,0.78,0.72,0.62,0.71,0.62,0.82
concrete,0.91,0.73,0.84,0.65,0.89,0.92
cpu_act,0.98,0.97,0.96,0.96,0.98,0.98
insurance,0.78,0.65,0.85,0.59,0.86,0.82
parkinsons,0.88,0.80,0.63,0.81,0.65,0.89
powerplant,0.93,0.94,0.94,0.92,0.94,0.96


In [121]:
pd.pivot_table(r_nfeat[r_nfeat['bit']==0], values='r2_score_val_true', aggfunc='max', index='dataset_name', columns=['model_name']).round(2).dropna()

model_name,figs,ft_transformer,random_forest,resnet,rf_plus,xgboost
dataset_name,,,,,,
abalone,0.34,0.47,0.51,0.45,0.53,0.50
airfoil,0.84,0.42,0.71,0.42,0.71,0.92
ca_housing,0.79,0.73,0.64,0.72,0.64,0.82
concrete,0.91,0.78,0.87,0.69,0.90,0.92
cpu_act,0.98,0.98,0.97,0.96,0.98,0.98
insurance,0.80,0.71,0.86,0.65,0.86,0.82
parkinsons,0.88,0.83,0.68,0.82,0.69,0.89
powerplant,0.94,0.94,0.94,0.93,0.94,0.96


In [122]:
pd.pivot_table(r_nfeat[r_nfeat['bit']==1], values='r2_score_val_true', aggfunc='max', index='dataset_name', columns=['model_name']).round(2).dropna()

model_name,figs,ft_transformer,random_forest,resnet,rf_plus,xgboost
dataset_name,,,,,,
abalone,0.34,0.45,0.51,0.46,0.53,0.50
airfoil,0.84,0.45,0.71,0.42,0.71,0.92
ca_housing,0.79,0.73,0.64,0.72,0.64,0.82
concrete,0.91,0.77,0.87,0.68,0.90,0.92
cpu_act,0.98,0.98,0.97,0.96,0.98,0.98
insurance,0.80,0.70,0.86,0.65,0.86,0.82
parkinsons,0.88,0.83,0.68,0.83,0.69,0.89
powerplant,0.94,0.94,0.94,0.93,0.94,0.96


In [129]:
from collections import defaultdict

# Create a defaultdict with default value 0
my_dict = defaultdict(lambda: 0)

# Add some key-value pairs
my_dict['a'] = 1
my_dict['b'] = 2

# Accessing existing keys
print(my_dict['a'])  # Output: 1
print(my_dict['b'])  # Output: 2

# Accessing a non-existing key
print(my_dict['c'])  # Output: 0


1
2
0


In [75]:
temp = r_feat.drop(columns=['n_epochs', 'n_epochs', 'gpu', 'size_interactions', 'use_cache', 'cat_mappings', 'task_type', 'save_dir', 
                     'seed', 'subsample_frac', 'featurizer_overlap', 'save_dir_unique'])

In [78]:
temp = temp.sort_values('r2_score_val_true', ascending = False)

In [80]:
temp['order'] = range(len(temp))

In [86]:
temp

,dataset_name,model_name,featurizer_name,featurizer_frac,depth,bit,max_depth,max_rules,max_trees,pre_interaction,pre_max_features,post_interaction,post_max_features,r2_score_train_true,r2_score_val_true,order
815,cpu_act,xgboost,featurizer,0.7,3,1,4,1,1,l0l2,0.0,l0l2,0.0,0.985843,0.976125,0
411,cpu_act,xgboost,featurizer,0.7,3,1,5,1,1,l0l2,0.0,l0l2,0.0,0.990971,0.975103,1
859,cpu_act,xgboost,featurizer,0.3,2,1,4,1,1,l0l2,0.0,l0l2,0.0,0.980086,0.973396,2
1222,cpu_act,xgboost,featurizer,0.7,3,0,4,1,1,l0l2,0.0,l0l2,0.0,0.986260,0.973360,3
149,cpu_act,xgboost,featurizer,0.7,2,1,4,1,1,l0l2,0.0,l0l2,0.0,0.980435,0.972745,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,airfoil,ft_transformer,featurizer,0.3,2,0,5,1,1,l0l2,0.0,l0l2,0.0,0.279403,0.124630,598
1101,airfoil,ft_transformer,featurizer,0.7,2,0,4,1,1,l0l2,0.0,l0l2,0.0,0.150728,0.109220,599
1217,airfoil,ft_transformer,featurizer,0.7,3,0,4,1,1,l0l2,0.0,l0l2,0.0,0.179102,0.102373,600
1135,airfoil,ft_transformer,featurizer,0.7,3,0,5,1,1,l0l2,0.0,l0l2,0.0,0.069315,0.066240,601


In [87]:
temp.groupby('featurizer_frac')['order'].mean()

featurizer_frac
0.3    298.091205
0.7    304.016892
Name: order, dtype: float64

In [85]:
temp.groupby('bit')['order'].mean()

bit
0    314.601504
1    290.264095
Name: order, dtype: float64

In [82]:
temp.groupby('depth')['order'].mean()

depth
2    303.382838
3    298.593333
Name: order, dtype: float64

In [70]:
pd.pivot_table(r_feat, values='r2_score_val_true', aggfunc='max', index='dataset_name', columns=['model_name']).round(2)

model_name,ft_transformer,random_forest,resnet,rf_plus,xgboost
dataset_name,,,,,
abalone,0.37,0.45,0.37,0.46,0.43
airfoil,0.48,0.60,0.46,0.64,0.86
ca_housing,0.69,0.61,0.66,0.67,0.73
concrete,0.67,0.77,0.63,0.83,0.86
cpu_act,0.97,0.96,0.95,0.97,0.98
insurance,0.77,0.86,0.63,0.86,0.82
miami_housing,0.80,0.78,0.79,0.79,0.88
parkinsons,0.74,0.83,0.72,0.83,0.92
powerplant,0.93,0.93,0.91,0.93,0.94


In [4]:
r = r[r['featurizer_name'] == 'featurizer']

In [45]:
distill1 = r[(r['featurizer_frac'] == 0.5) & (r['distiller_name'].isin(['figs', 'ft_distill'])) & (r['featurizer_overlap'])]

In [48]:
distill1

,dataset_name,subsample_frac,seed,save_dir,model_name,distiller_name,featurizer_name,featurizer_frac,featurizer_overlap,depth,...,distiller_r2_score_train_teacher,distiller_r2_score_test_teacher,teacher_f_r2_score_train_true,teacher_f_r2_score_test_true,distiller_f_r2_score_train_true,distiller_f_r2_score_test_true,distiller_f_r2_score_train_teacher,distiller_f_r2_score_test_teacher,r2_score_train_true,r2_score_test_true
5,powerplant,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,ft_distill,featurizer,0.5,1,2,...,0.997677,0.997956,0.902359,0.904352,0.902233,0.903458,0.998407,0.998373,NaN,NaN
11,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,ft_distill,featurizer,0.5,1,2,...,0.902463,0.903155,0.630980,0.586971,0.604681,0.586363,0.915640,0.917754,NaN,NaN
15,parkinsons,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,featurizer,0.5,1,3,...,0.944363,0.936908,0.863653,0.835765,0.867774,0.845302,0.976547,0.962567,NaN,NaN
25,parkinsons,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,ft_distill,featurizer,0.5,1,2,...,0.699434,0.609549,0.838977,0.704210,0.715560,0.701452,0.748076,0.697684,NaN,NaN
32,powerplant,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,figs,featurizer,0.5,1,2,...,0.998280,0.997824,0.906390,0.909045,0.905777,0.907908,0.998301,0.997996,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,parkinsons,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,figs,featurizer,0.5,1,2,...,0.922620,0.791816,0.836900,0.736941,0.864377,0.838808,0.922809,0.800051,NaN,NaN
637,parkinsons,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,figs,featurizer,0.5,1,3,...,0.838008,0.644061,0.757673,0.596410,0.793380,0.747339,0.835230,0.657103,NaN,NaN
664,powerplant,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,ft_distill,featurizer,0.5,1,2,...,0.984398,0.985448,0.904981,0.907552,0.900422,0.903417,0.983954,0.985517,NaN,NaN
669,powerplant,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,featurizer,0.5,1,3,...,0.998959,0.998878,0.930110,0.932350,0.928359,0.930948,0.997215,0.996873,NaN,NaN


In [59]:
pd.pivot_table(distill1[distill1['depth']==2], values='distiller_f_r2_score_test_true', aggfunc='mean', index='dataset_name', columns=['model_name', 'distiller_name']).round(2)

model_name     ft_transformer            resnet           
distiller_name           figs ft_distill   figs ft_distill
dataset_name                                              
airfoil                  0.64       0.64   0.63       0.62
ca_housing               0.58       0.57   0.58       0.58
parkinsons               0.85       0.68   0.83       0.66
powerplant               0.90       0.90   0.91       0.90

In [52]:
pd.pivot_table(distill1, values='distiller_r2_score_test_true', aggfunc='max', index='dataset_name', columns=['model_name', 'distiller_name']).round(2)

model_name     ft_transformer            resnet           
distiller_name           figs ft_distill   figs ft_distill
dataset_name                                              
airfoil                  0.72       0.69   0.69       0.66
ca_housing               0.68       0.65   0.68       0.66
parkinsons               0.85       0.72   0.84       0.69
powerplant               0.93       0.93   0.94       0.93

In [23]:
train1 = r[(r['featurizer_frac'] == 0.5) & (r['model_name'].isin(['figs', 'ft_distill'])) & (r['featurizer_overlap'])]
train0 = r[(r['featurizer_frac'] == 0.5) & (r['model_name'].isin(['figs', 'ft_distill'])) & (r['featurizer_overlap']==0)]

In [55]:
pd.pivot_table(train1[train1['depth'] ==2], values='r2_score_test_true', aggfunc='mean', index='dataset_name', columns=['model_name']).round(2)

model_name,figs,ft_distill
dataset_name,,
abalone,0.46,0.46
airfoil,0.80,0.66
ca_housing,0.60,0.59
concrete,0.45,0.40
cpu_act,0.97,0.96
insurance,0.82,0.86
miami_housing,0.84,0.80
parkinsons,0.88,0.67
powerplant,0.91,0.91


In [42]:
train1['bit']==0

3      False
7      False
19     False
21     False
22     False
       ...  
650    False
659    False
668    False
687    False
689    False
Name: bit, Length: 72, dtype: bool

In [40]:
pd.pivot_table(train1[(train1['depth'] == 2) & (train1['bit']==0)], values='r2_score_test_true', aggfunc='max', index='dataset_name', columns=['model_name']).round(2)

model_name
dataset_name


In [36]:
pd.pivot_table(train1[train1['depth'] == 3], values='r2_score_test_true', aggfunc='max', index='dataset_name', columns=['model_name']).round(2)

model_name,figs,ft_distill
dataset_name,,
abalone,0.47,0.48
airfoil,0.78,0.66
ca_housing,0.70,0.66
concrete,0.69,0.54
cpu_act,0.96,0.96
insurance,0.72,0.86
miami_housing,0.80,0.80
parkinsons,0.88,0.67
powerplant,0.94,0.93


In [24]:
train1 = train1.dropna(axis = 1)
train0 = train0.dropna(axis = 1)

In [30]:
pd.pivot_table(train1, values='r2_score_test_true', aggfunc='median', index='dataset_name', columns=['model_name']).round(2)

model_name,figs,ft_distill
dataset_name,,
abalone,0.47,0.46
airfoil,0.79,0.64
ca_housing,0.65,0.62
concrete,0.57,0.40
cpu_act,0.97,0.96
insurance,0.77,0.86
miami_housing,0.82,0.80
parkinsons,0.88,0.65
powerplant,0.93,0.92


In [29]:
pd.pivot_table(train0, values='r2_score_test_true', aggfunc='mean', index='dataset_name', columns=['model_name']).round(2)

model_name,figs,ft_distill
dataset_name,,
abalone,0.42,0.43
airfoil,0.71,0.50
ca_housing,0.65,0.61
concrete,0.52,0.30
cpu_act,0.97,0.95
insurance,0.67,0.84
miami_housing,0.45,0.70
parkinsons,0.87,0.40
powerplant,0.92,0.92


In [ ]:
r_01 =r.dropna(subset=['distiller_f_r2_score_train_true'])

In [ ]:
r_01

In [ ]:
r_01['model_name'].isin(['figs', 'ft_distill'])

In [ ]:
r_01[(r_01['featurizer_frac'] == 0.5) & (r_01['model_name'].isin(['figs', 'ft_distill']))]

In [ ]:
r_01['featurizer_frac'] == 0.5 & r_01['model_name'] in ['figs', 'ft_distill']

In [ ]:
r_train = r_train.dropna(axis = 1)

In [ ]:
r_train['r2_score_test_true'] = r_train['r2_score_test_true'].round(2)

In [ ]:
pd.pivot_table(r_train, values='r2_score_test_true', aggfunc='median', index='dataset_name', columns=['model_name'])

In [ ]:
figs = r[r['distiller_name'] == 'figs']
pd.pivot_table(figs, values='distiller_r2_score_test_teacher', aggfunc='median', index='dataset_name', columns=['distiller_name', 'model_name'])

In [ ]:
pd.pivot_table(figs, values='teacher_r2_score_test_true', aggfunc='median', index='dataset_name', columns=['distiller_name', 'model_name'])

In [ ]:
ft_distill = r[r['distiller_name'] == 'ft_distill']
pd.pivot_table(ft_distill, values='distiller_r2_score_test_teacher', aggfunc='median', index='dataset_name', columns=['distiller_name', 'model_name'])

In [ ]:
pd.pivot_table(ft_distill, values='teacher_r2_score_test_true', aggfunc='median', index='dataset_name', columns=['distiller_name', 'model_name'])

In [ ]:
r.sort_values('teacher_r2_score_test_true', ascending=False)

In [ ]:
# group using these experiment hyperparams when averaging over random seeds
ravg = imodelsx.process_results.average_over_seeds(
    r, experiment_filename, key_to_average_over='seed'
)

# apply cross validation
ravg_cv = (
    ravg
    .sort_values(by='distiller_r2_score_test_true', ascending=False)
    .groupby(by=['model_name', 'dataset_name'])
    .max()
    .reset_index()
)
ravg_cv

In [ ]:
# , hue='dataset_name')
sns.barplot(data=ravg_cv, x='model_name', y='accuracy_test')
plt.show()

Load an individual model

In [ ]:
run_args = r.iloc[0]
model = joblib.load(join(run_args.save_dir_unique, 'model.pkl'))
model